In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/"My Drive"/"Master Thesis"/"Model"

/content/drive/My Drive/Master Thesis/Model


Importing dependencies numpy and keras

In [0]:
import numpy, os
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, LSTM, SimpleRNN, Embedding
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

生成時間序列資料 load traffic data

In [4]:
data_dir = "../Data"

x, y = None, None   # 初始化

for env_folder in os.listdir(data_dir):
    print(env_folder)
    # 檢查.npy檔
    if os.path.isfile(os.path.join(data_dir, env_folder, 'X.npy')) and os.path.isfile(os.path.join(data_dir, env_folder, 'Y.npy')):
        env_x = np.load(os.path.join(data_dir, env_folder, 'X.npy'), allow_pickle=True)
        env_y = np.load(os.path.join(data_dir, env_folder, 'Y.npy'), allow_pickle=True)    # 從 .npy 檔載入已經算好的數據
        print("env_x.shape:", env_x.shape)
        print("env_y.shape:", env_y.shape)
        if x is None or y is None:
            x = env_x.copy()
            y = env_y.copy()
        else:
            x = np.vstack([x, env_x])
            y = np.vstack([y, env_y])

print("x.shape:", x.shape)
print("y.shape:", y.shape)

01-section
env_x.shape: (5017, 5, 36)
env_y.shape: (5017, 5, 2)
02-section
env_x.shape: (3294, 5, 36)
env_y.shape: (3294, 5, 2)
x.shape: (8311, 5, 36)
y.shape: (8311, 5, 2)


In [5]:
x[0, 0]

array([5.1000000000000005, 1.500000000000001, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, 'small bus (parking)',
       39.47000000000001, -2.93, 39.57860280505113, -5.1000000000000005,
       -1.500000000000001, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, 3.728, nan, nan, -5.482000000000001], dtype=object)

Input Data 標準化

Some parameters

In [0]:
BATCH_START = 0
# TIME_STEPS = X_train.shape[1]
TIME_STEPS = x.shape[1]
BATCH_SIZE = 20
# INPUT_SIZE = X_train.shape[2]
INPUT_DIM = x.shape[2]
# OUTPUT_SIZE = y_train.shape[2]
OUTPUT_DIM = y.shape[2]
UNITS = 20      # hidden layer unit數
LR = 1.25E-4
EPOCHS = 50

Constructing the LSTM model

In [0]:
model.add(
    LSTM(
        300,
        input_shape=(TIME_STEPS, INPUT_SIZE),
        return_sequences=False
    )
)
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(
    Dense(
        OUTPUT_SIZE,
        activation=None
    )
)
# This returns a tensor
inputs = Input(shape=(TIME_STEPS, INPUT_DIM))

#
embedded_input = Embedding(output_dim=10)(inputs)
lstm_output = LSTM(units=UNITS)(embedded_input)

##
fc_output = 


predictions = Dense(10, activation='softmax')(output_2)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

In [0]:
from keras.utils import plot_model

plot_model(model, to_file='./model.png')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 300)            404400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 300)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 602       
Total params: 1,126,202
Trainable params: 1,126,202
Non-trainable params: 0
_________________________________________________________________


Loss function: x, y座標的 loss

In [0]:
from keras import backend as K

def loss_function(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))    # RMSE

In [0]:
model.compile(
    loss=loss_function, 
    optimizer=Adam(LR)
)

Training log file

In [0]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger(
    filename="./model_test.log",
    separator=',',
    append=True
)

Training

In [0]:
history = model.fit(
    X=X,
    y=y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[csv_logger],
    validation_split=0.33
)